<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-24 11:12:13
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.40 C
-------------------
Today PnL: -1.14 L (-0.81%)
Current PnL: -34.13 L (-21.34%)
CY Booked + Current PnL: -18.45 L (-11.53%)
-------------------
Total profit:  1.34 L
Total loss:  -35.47 L
-------------------
Total Booked + Current PnL: 8.53 L (6.41%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.18%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.01 C (71.81%)
Deployed:  1.33 C
Current:  1.40 C
CAGR/XIRR %: 2.74%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.03,3.37,17.61,21.57,17835.0,3300.0,101280.0,397.04,-11.02,58.0,H-SC,6.20,167.0,0.19,0.73,47.80,XR,ATH,FINANCE
18,COALINDIA,-0.45,4.29,12.83,17.67,20132.0,6456.0,156917.0,478.51,26.26,54.0,L-LC,11.51,185.0,0.32,1.14,25.97,XY25,ATH,MINING
77,TTKPRESTIG,0.98,-30.46,43.93,0.09,30788.0,-30692.0,70085.0,770.00,62.65,33.0,M-SC,10.48,250.0,-1.00,0.51,0.98,OX40N,NTT,DURABLES
36,ICICIGI,-1.18,2.28,16.68,19.34,34138.0,4556.0,204665.0,2252.93,-13.95,56.0,X-MC,1.68,66.0,0.13,1.48,18.58,X40,ATH,INSURANCE
51,MEDANTA,-1.66,-4.26,29.60,24.08,35976.0,-5410.0,121540.0,1486.00,-4.88,51.0,X-SC,9.36,87.0,-0.15,0.88,11.32,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,2.46,-14.41,124.82,92.42,297587.0,-40146.0,238413.0,400.0,104.74,74.0,H-SC,6.50,174.0,-0.13,1.73,32.67,XY24,NTT,BANKS
77,TTKPRESTIG,0.98,-30.46,43.93,0.09,30788.0,-30692.0,70085.0,770.0,62.65,33.0,M-SC,10.48,250.0,-1.00,0.51,0.98,OX40N,NTT,DURABLES
0,5PAISA,0.73,-38.64,83.99,12.90,112611.0,-84419.0,134077.0,593.0,97.12,43.0,H-SC,7.44,170.0,-0.75,0.97,11.48,OX40N,NTT,FINANCE
41,INDUSINDBK,0.66,-31.65,94.43,32.89,49832.0,-24436.0,52771.0,1800.0,-348.60,54.0,L-MC,11.48,258.0,-0.49,0.38,45.33,XR,NTT,BANKS
82,VOLTAS,0.65,21.12,19.88,45.20,46167.0,40488.0,232230.0,1856.0,38.62,65.0,X-MC,7.40,79.0,0.88,1.68,29.35,XY25,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HCLTECH,-4.41,-11.53,39.97,23.83,85551.0,-27886.0,214038.0,1908.19,-6.95,19.0,X-LC,3.82,8.0,-0.33,1.55,2.69,X40,ATH,IT
25,EASEMYTRIP,-4.05,-51.60,201.03,45.70,172037.0,-91237.0,85578.0,26.40,-11.26,64.0,M-SC,18.83,204.0,-0.53,0.62,41.00,XY24,NTT,TRAVEL
71,TANLA,-3.61,-49.63,336.15,119.69,458455.0,-134377.0,136384.0,1943.92,-49.08,36.0,H-SC,19.07,147.0,-0.29,0.99,7.74,AR,ATH,IT
42,INFY,-3.41,-12.71,53.80,34.24,146933.0,-39779.0,273109.0,1972.00,-31.42,18.0,X-LC,6.31,3.0,-0.27,1.98,0.00,X40,NTT,IT
72,TATAELXSI,-3.29,-34.30,100.94,32.03,87435.0,-45214.0,86621.0,9161.00,-17.84,28.0,H-SC,9.29,158.0,-0.52,0.63,0.00,OX40N,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-1.55,2.90,102.06,107.91,155049.0,4275.0,151919.0,1641.55,-7.66,58.0,M-SC,7.93,220.0,0.03,1.10,14.29,OX40N,ATH,CABLES
77,TTKPRESTIG,0.98,-30.46,43.93,0.09,30788.0,-30692.0,70085.0,770.00,62.65,33.0,M-SC,10.48,250.0,-1.00,0.51,0.98,OX40N,NTT,DURABLES
66,SIS,-0.22,-31.31,76.53,21.26,58369.0,-34762.0,76270.0,528.00,1790.16,32.0,H-SC,4.43,168.0,-0.60,0.55,5.15,OX40N,NTT,MISC
40,INDIGOPNTS,-0.27,-32.64,48.48,0.02,57007.0,-56970.0,117589.0,1408.00,79.13,28.0,M-SC,10.83,222.0,-1.00,0.85,2.28,OX40N,NTT,PAINTS
48,KANSAINER,-0.31,-33.58,70.84,13.47,126887.0,-90549.0,179118.0,340.00,-42.90,27.0,H-SC,11.72,162.0,-0.71,1.30,0.17,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.65,21.12,19.88,45.2,46167.0,40488.0,232230.0,1856.0,38.62,65.0,X-MC,7.4,79.0,0.88,1.68,29.35,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.45,4.29,12.83,17.67,20132.0,6456.0,156917.0,478.51,26.26,54.0,L-LC,11.51,185.0,0.32,1.14,25.97,XY25,ATH,MINING
26,FINCABLES,-1.55,2.90,102.06,107.91,155049.0,4275.0,151919.0,1641.55,-7.66,58.0,M-SC,7.93,220.0,0.03,1.10,14.29,OX40N,ATH,CABLES
50,MASFIN,-1.03,3.37,17.61,21.57,17835.0,3300.0,101280.0,397.04,-11.02,58.0,H-SC,6.20,167.0,0.19,0.73,47.80,XR,ATH,FINANCE
1,ABB,0.13,8.71,33.91,45.57,96440.0,22781.0,284400.0,7934.00,-30.12,64.0,H-MC,7.63,125.0,0.24,2.06,26.29,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.13,8.71,33.91,45.57,96440.0,22781.0,284400.0,7934.00,-30.12,64.0,H-MC,7.63,125.0,0.24,2.06,26.29,AR,NTT,ELECTRICAL
50,MASFIN,-1.03,3.37,17.61,21.57,17835.0,3300.0,101280.0,397.04,-11.02,58.0,H-SC,6.20,167.0,0.19,0.73,47.80,XR,ATH,FINANCE
26,FINCABLES,-1.55,2.90,102.06,107.91,155049.0,4275.0,151919.0,1641.55,-7.66,58.0,M-SC,7.93,220.0,0.03,1.10,14.29,OX40N,ATH,CABLES
39,INDIAMART,-0.69,-5.38,118.47,106.72,138262.0,-6630.0,116706.0,4810.62,-57.23,49.0,H-SC,7.97,140.0,-0.05,0.85,16.75,AR,ATH,MISC
85,ZYDUSLIFE,-0.29,-4.53,42.07,35.63,84170.0,-9503.0,200071.0,1286.17,-17.12,51.0,H-MC,5.09,120.0,-0.11,1.45,11.38,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-3.41,-12.71,53.80,34.24,146933.0,-39779.0,273109.0,1972.00,-31.42,18.0,X-LC,6.31,3.0,-0.27,1.98,0.00,X40,NTT,IT
33,HCLTECH,-4.41,-11.53,39.97,23.83,85551.0,-27886.0,214038.0,1908.19,-6.95,19.0,X-LC,3.82,8.0,-0.33,1.55,2.69,X40,ATH,IT
8,AWL,-1.53,-37.55,151.73,57.19,309001.0,-122475.0,203652.0,485.00,-64.34,21.0,X-SC,17.88,81.0,-0.40,1.48,0.00,XY24,NTT,FMCG
73,TCS,-2.91,-25.52,65.93,23.58,212436.0,-110415.0,322214.0,4311.59,-35.43,23.0,X-LC,6.95,2.0,-0.52,2.34,0.00,X40,ATH,IT
12,BATAINDIA,-1.15,-48.20,161.02,35.20,107318.0,-62021.0,66649.0,2096.00,-12.64,30.0,X-SC,21.87,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,0.11,4.19,15.91,20.77,38815.0,9821.0,243965.0,1644.00,-7.55,58.0,X-MC,0.56,64.0,0.25,1.77,11.91,X40N,NTT,BREWERIES
21,DABUR,0.01,0.57,42.88,43.70,106979.0,1416.0,249484.0,735.00,-8.76,52.0,X-MC,1.35,70.0,0.01,1.81,15.71,XY24,BTT,FMCG
36,ICICIGI,-1.18,2.28,16.68,19.34,34138.0,4556.0,204665.0,2252.93,-13.95,56.0,X-MC,1.68,66.0,0.13,1.48,18.58,X40,ATH,INSURANCE
65,SIEMENS,-0.36,-11.59,41.97,25.51,69049.0,-21575.0,164520.0,4671.50,38.15,61.0,H-LC,2.43,51.0,-0.31,1.19,21.24,AR,ATH,ELECTRICAL
34,HINDUNILVR,0.22,-8.56,24.31,13.67,57143.0,-22003.0,235060.0,2922.00,-32.06,49.0,X-LC,2.76,9.0,-0.39,1.70,10.97,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,-0.13,-25.35,108.26,55.47,196623.0,-61665.0,181621.0,181.5,-36.04,31.0,X-MC,17.71,60.0,-0.31,1.32,0.0,X40N,ATH,FINANCE
12,BATAINDIA,-1.15,-48.20,161.02,35.20,107318.0,-62021.0,66649.0,2096.0,-12.64,30.0,X-SC,21.87,91.0,-0.58,0.48,0.0,X40,NTT,FOOTWEAR
8,AWL,-1.53,-37.55,151.73,57.19,309001.0,-122475.0,203652.0,485.0,-64.34,21.0,X-SC,17.88,81.0,-0.40,1.48,0.0,XY24,NTT,FMCG
42,INFY,-3.41,-12.71,53.80,34.24,146933.0,-39779.0,273109.0,1972.0,-31.42,18.0,X-LC,6.31,3.0,-0.27,1.98,0.0,X40,NTT,IT
58,RELAXO,-1.30,-54.39,233.95,52.31,154884.0,-78956.0,66204.0,1176.0,-51.41,36.0,X-SC,14.03,92.0,-0.51,0.48,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-1.66,-32.55,111.79,42.84,49013.0,-21162.0,43844.0,424.00,-55.92,43.0,X-SC,17.90,80.0,-0.43,0.32,5.46,XY24,NTT,MISC
12,BATAINDIA,-1.15,-48.20,161.02,35.20,107318.0,-62021.0,66649.0,2096.00,-12.64,30.0,X-SC,21.87,91.0,-0.58,0.48,0.00,X40,NTT,FOOTWEAR
58,RELAXO,-1.30,-54.39,233.95,52.31,154884.0,-78956.0,66204.0,1176.00,-51.41,36.0,X-SC,14.03,92.0,-0.51,0.48,0.00,X40N,NTT,FOOTWEAR
51,MEDANTA,-1.66,-4.26,29.60,24.08,35976.0,-5410.0,121540.0,1486.00,-4.88,51.0,X-SC,9.36,87.0,-0.15,0.88,11.32,XY24,NTT,HEALTHCARE
35,HONAUT,0.03,-22.44,84.79,43.32,107107.0,-36552.0,126320.0,58357.33,-31.88,44.0,X-SC,11.38,90.0,-0.34,0.92,2.57,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.37,-14.61,39.42,19.06,114508.0,-49683.0,290483.0,452.00,-53.09,48.0,X-LC,17.75,1.0,-0.43,2.11,5.18,X40,NTT,FMCG
73,TCS,-2.91,-25.52,65.93,23.58,212436.0,-110415.0,322214.0,4311.59,-35.43,23.0,X-LC,6.95,2.0,-0.52,2.34,0.00,X40,ATH,IT
42,INFY,-3.41,-12.71,53.80,34.24,146933.0,-39779.0,273109.0,1972.00,-31.42,18.0,X-LC,6.31,3.0,-0.27,1.98,0.00,X40,NTT,IT
75,TMPV,-0.58,-16.06,58.84,33.33,160922.0,-52309.0,273491.0,600.00,-82.53,57.0,X-LC,3.00,4.0,-0.33,1.98,11.81,XY24,NTT,AUTO
33,HCLTECH,-4.41,-11.53,39.97,23.83,85551.0,-27886.0,214038.0,1908.19,-6.95,19.0,X-LC,3.82,8.0,-0.33,1.55,2.69,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.66,-31.65,94.43,32.89,49832.0,-24436.0,52771.0,1800.00,-348.60,54.0,L-MC,11.48,258.0,-0.49,0.38,45.33,XR,NTT,BANKS
67,SONACOMS,-1.41,-8.57,52.13,39.09,48214.0,-8671.0,92488.0,804.02,-27.49,59.0,M-MC,9.57,193.0,-0.18,0.67,30.84,AR,ATH,AUTO
50,MASFIN,-1.03,3.37,17.61,21.57,17835.0,3300.0,101280.0,397.04,-11.02,58.0,H-SC,6.20,167.0,0.19,0.73,47.80,XR,ATH,FINANCE
18,COALINDIA,-0.45,4.29,12.83,17.67,20132.0,6456.0,156917.0,478.51,26.26,54.0,L-LC,11.51,185.0,0.32,1.14,25.97,XY25,ATH,MINING
37,ICICIPRULI,-0.35,9.52,19.36,30.73,41772.0,18758.0,215763.0,790.00,-19.89,58.0,X-MC,4.05,78.0,0.45,1.57,23.50,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-4.05,-51.60,201.03,45.70,172037.0,-91237.0,85578.0,26.4,-11.26,64.0,M-SC,18.83,204.0,-0.53,0.62,41.00,XY24,NTT,TRAVEL
1,ABB,0.13,8.71,33.91,45.57,96440.0,22781.0,284400.0,7934.0,-30.12,64.0,H-MC,7.63,125.0,0.24,2.06,26.29,AR,NTT,ELECTRICAL
7,ATULAUTO,-1.11,-18.08,77.68,45.57,125453.0,-35632.0,161500.0,844.0,3494.59,53.0,M-SC,4.49,248.0,-0.28,1.17,22.22,XY24,NTT,AUTO
11,BANDHANBNK,2.46,-14.41,124.82,92.42,297587.0,-40146.0,238413.0,400.0,104.74,74.0,H-SC,6.50,174.0,-0.13,1.73,32.67,XY24,NTT,BANKS
83,WHIRLPOOL,-0.28,-26.09,146.12,81.92,123986.0,-29946.0,84852.0,2270.0,-56.73,61.0,M-SC,26.27,210.0,-0.24,0.62,19.19,XR,NTT,DURABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.65
1,25,45.48
2,50,77.44


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.09
MC    30.25
LC    24.66
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.00
X40      23.89
X40N     14.85
AR        9.01
XY25      8.86
XR        8.45
OX40N     7.10
SR        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.78
H-SC    22.12
X-LC    21.31
X-SC    11.55
M-SC    10.68
H-MC     5.01
M-MC     2.08
H-LC     1.19
L-LC     1.14
M-LC     1.02
L-SC     0.74
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.65,-14.42,53.12
FINANCE,13.13,-15.26,60.62
IT,11.20,-42.05,113.25
MISC,6.90,-36.68,92.59
ELECTRICAL,6.32,-7.64,46.92
PAINTS,5.05,-32.50,52.19
INSURANCE,4.84,-1.69,36.49
PHARMA,4.11,-3.82,36.49
AUTO,3.82,-18.32,63.43


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3403691.0,21
X40,1426334.0,15
AR,1329660.0,10
XR,1328149.0,12
X40N,1075890.0,10
OX40N,811544.0,10
XY25,395478.0,6
SR,304065.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3743677.0,24
M-SC,1446213.0,14
X-MC,1372258.0,15
X-LC,1333749.0,12
X-SC,1211868.0,10
H-MC,387570.0,3
L-SC,189254.0,2
M-LC,151686.0,1
M-MC,99523.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1267614.0      6
           AR         956906.0      5
           XR         855135.0      7
M-SC       XY24       837503.0      6
X-LC       X40        700899.0      6
X-SC       XY24       622606.0      4
X-MC       X40        543540.0      7
           X40N       435265.0      4
X-SC       X40N       407367.0      4
H-SC       OX40N      359957.0      4
M-SC       OX40N      355603.0      5
H-SC       SR         304065.0      2
X-LC       XY24       273973.0      2
           X40N       233258.0      2
H-MC       XY24       206960.0      1
X-MC       XY25       198418.0      2
           XY24       195035.0      2
X-SC       X40        181895.0      2
H-MC       AR         180610.0      2
M-SC       XR         178226.0      2
M-LC       XR         151686.0      1
X-LC       XY25       125619.0      2
L-SC       OX40N       95984.0      1
           XR          93270.0      1
M-SC       AR          74881.0      1
H-LC       AR          69049.0      1
M-MC       XY25        51309.0      1
L-MC       XR          49832.0      1
M-MC       AR          48214.0      1
L-LC       XY25        20132.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
